In [1]:
from time import time
start = time()

In [2]:
import os
GPUs = ','.join([str(i) for i in range(0,16)])
num_gpus = len(GPUs.split(','))
os.environ['CUDA_VISIBLE_DEVICES'] = GPUs
os.environ["RAPIDS_NO_INITIALIZE"] = "1" #Vjawa: Added

In [3]:
import dask, dask_cudf
from dask.distributed import Client, wait
from dask_cuda import LocalCUDACluster
import subprocess
import xgboost as xgb

def enable_spilling():
    import cudf
    cudf.set_option("spill", True)
    
def get_ip():
    cmd = "hostname --all-ip-addresses"
    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
    output, error = process.communicate()
    IPADDR = str(output.decode()).split()[0]
    print(IPADDR)
    return IPADDR

def get_cluster():
    ip = get_ip()
    cluster = LocalCUDACluster(ip=ip,
                               protocol="ucx", 
                               rmm_pool_size="31GB")
    client = Client(cluster)
    
    #client.run(enable_spilling)
    return client


In [4]:

client = get_cluster()
client

10.120.104.12


2023-04-13 17:14:28,954 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10093/worker-y006cwx3', purging
2023-04-13 17:14:28,955 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10093/worker-r0x1yosn', purging
2023-04-13 17:14:28,955 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 17:14:28,955 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 17:14:28,955 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 17:14:28,955 - distributed.preloading - INFO - Import preload module: dask_cuda.initialize
2023-04-13 17:14:28,955 - distributed.preloading - INFO - Creating preload: dask_cuda.initialize
2023-04-13 17:14:28,955 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-worker-space-10093/worker-qhvopipz', purging
2023-04-13 17:14:28,955 - distributed.preloading - INFO - 

Connection method: Cluster object,Cluster type: dask_cuda.LocalCUDACluster
Dashboard: http://10.120.104.12:8787/status,
Dashboard: http://10.120.104.12:8787/status,Workers: 16
Total threads: 16,Total memory: 1.48 TiB
Status: running,Using processes: True
Comm: ucx://10.120.104.12:38755,Workers: 16
Dashboard: http://10.120.104.12:8787/status,Total threads: 16
Started: Just now,Total memory: 1.48 TiB
Comm: ucx://10.120.104.12:56907,Total threads: 1
Dashboard: http://10.120.104.12:44547/status,Memory: 94.41 GiB
Nanny: ucx://10.120.104.12:48541,


In [5]:
%%time

users = dask_cudf.read_parquet('/raid/otto/Otto-Comp/pqs/train_v152_*.pq').persist()#,split_row_groups=True)
users.head()

CPU times: user 2.57 s, sys: 1.62 s, total: 4.19 s
Wall time: 12.4 s


,user,item,pos,wgt,clicks,score_1,score_3,score_4,score_5,score_6,...,cart_repeat9,buy_ratio10,count_item10,unique_item10,hour_mean10,day_mean10,repeat10,buy_ratio_std10,hour_std10,day_std10
0,11098528,11830,0,-1.000,0,-1.0,-1.0,-1.0,-1.0,-1.0,...,1.965084,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,11098528,588923,1,14.500,0,2.0,1.0,-3.0,2.0,4.0,...,2.031830,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,11098528,884502,2,12.485,0,1.0,1.0,-3.0,1.0,2.0,...,1.925403,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,11098528,1182614,3,12.470,0,1.0,1.0,-3.0,1.0,1.5,...,1.904306,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,11098528,571762,4,11.990,0,1.0,1.0,-3.0,1.0,2.0,...,1.953917,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [6]:
%%time
users.shape[0].compute()

CPU times: user 354 ms, sys: 185 ms, total: 539 ms
Wall time: 1.58 s


180125100

In [7]:
users.npartitions

182

In [8]:
%%time
users.memory_usage().sum().compute()/2**30

CPU times: user 465 ms, sys: 101 ms, total: 567 ms
Wall time: 614 ms


105.77028216049075

In [9]:
VER = 186
USE = 'clicks'

In [10]:
FEATURES = users.columns[2:]
TARS = [USE]
FEATURES = [f for f in FEATURES if f not in TARS]
print(len(FEATURES))
print( FEATURES)
print(TARS)

152
['pos', 'wgt', 'score_1', 'score_3', 'score_4', 'score_5', 'score_6', 'score_7', 'score_10', 'score_11', 'score_12', 'score_13', 'score_30', 'score_14', 'score_15', 'score_16', 'score_17', 'score_18', 'score_19', 'score_20', 'score_21', 'score_22', 'score_23', 'score_24', 'score_25', 'score_26', 'buy_ratio_5', 'count_item_5', 'count_user_5', 'repeat_5', 'buy_ratio_std_5', 'item_day_std_5', 'user_day_std_5', 'buy_ratio_0_6', 'buy_ratio_1_6', 'buy_ratio_2_6', 'buy_ratio_3_6', 'buy_ratio_4_6', 'buy_ratio_5_6', 'buy_ratio_6_6', 'count_item_0_6', 'count_item_1_6', 'count_item_2_6', 'count_item_3_6', 'count_item_4_6', 'count_item_5_6', 'count_item_6_6', 'count_user_0_6', 'count_user_1_6', 'count_user_2_6', 'count_user_3_6', 'count_user_4_6', 'count_user_5_6', 'count_user_6_6', 'item_day_std_6', 'user_day_std_6', 'buy_ratio', 'count_item', 'count_user', 'hour_mean', 'day_mean', 'repeat', 'buy_ratio_std', 'hour_std', 'day_std', 'buy_ratio3', 'count_item3', 'count_user3', 'hour_mean3', 'day

In [11]:
%%time

dtrain = xgb.dask.DaskQuantileDMatrix(client, users[FEATURES].astype('float32'), users['clicks'].astype('float32'))

CPU times: user 324 ms, sys: 36.9 ms, total: 361 ms
Wall time: 360 ms


In [12]:
FOLDS = 5
SEED = 42

LR = 0.1

# XGB MODEL PARAMETERS
xgb_parms = { 
    'max_depth':4, 
    'learning_rate':LR, 
    'subsample':0.7,
    'colsample_bytree':0.5, 
    'eval_metric':'map',
    'objective':'binary:logistic',
    'scale_pos_weight':8,
    'tree_method':'gpu_hist',
    'predictor':'gpu_predictor',
    'random_state':SEED
}

In [13]:
%%time

output = xgb.dask.train(
        client,
        xgb_parms,
        dtrain,
        num_boost_round=100,
        evals=[(dtrain, "train")],
    )

/raid/data/mambaforge/envs/rapids-23.04/lib/python3.10/site-packages/xgboost/dask.py:856: RuntimeWarning: coroutine 'Client._wait_for_workers' was never awaited
  client.wait_for_workers(n_workers)
INFO:distributed.scheduler:Receive client connection: Client-worker-642e61c2-da59-11ed-a1e6-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38755
INFO:distributed.scheduler:Receive client connection: Client-worker-642f54c8-da59-11ed-a1e8-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38755
INFO:distributed.scheduler:Receive client connection: Client-worker-643e8bdc-da59-11ed-a1d7-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38755
INFO:distributed.scheduler:Receive client connection: Client-worker-643e9c78-da59-11ed-a1e4-5cff35c18da6
INFO:distributed.core:Starting established connection to ucx://10.120.104.12:38755
INFO:distributed.scheduler:Receive client connecti

[0]	train-map:0.16624
[1]	train-map:0.17607
[2]	train-map:0.17614
[3]	train-map:0.17474
[4]	train-map:0.18265
[5]	train-map:0.18325
[6]	train-map:0.18281
[7]	train-map:0.18294
[8]	train-map:0.18395
[9]	train-map:0.18455
[10]	train-map:0.18584
[11]	train-map:0.18690
[12]	train-map:0.18652
[13]	train-map:0.18609
[14]	train-map:0.18637
[15]	train-map:0.18676
[16]	train-map:0.18687
[17]	train-map:0.18738
[18]	train-map:0.18756
[19]	train-map:0.18797
[20]	train-map:0.18898
[21]	train-map:0.18916
[22]	train-map:0.18969
[23]	train-map:0.19000
[24]	train-map:0.19003
[25]	train-map:0.19024
[26]	train-map:0.19018
[27]	train-map:0.19016
[28]	train-map:0.19050
[29]	train-map:0.19086
[30]	train-map:0.19116
[31]	train-map:0.19159
[32]	train-map:0.19206
[33]	train-map:0.19261
[34]	train-map:0.19341
[35]	train-map:0.19358
[36]	train-map:0.19389
[37]	train-map:0.19480
[38]	train-map:0.19520
[39]	train-map:0.19553
[40]	train-map:0.19589
[41]	train-map:0.19613
[42]	train-map:0.19630
[43]	train-map:0.1965

In [14]:
duration = time()-start
duration

79.26879787445068